In [1]:
%matplotlib widget
from IPython.display import display, Markdown
import os
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model._base import LinearModel
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from rega3 import export_estimator

polyfeat = PolynomialFeatures(degree=2)
linreg = LinearRegression()
pipe = Pipeline([('polyfeat', polyfeat), ('linreg', linreg)])

In [2]:
display(Markdown('### Load Data\n---'))
df = pd.read_csv('ChosunOne_ArtilleryData.csv')
X = df.drop(['Firing Angle', 'Firing Mode', 'Weapon Platform'], axis=1).values
y = df['Firing Angle'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
print('X.shape:', X.shape, sep='\t')
print('X_train.shape:', X_train.shape, sep='\t')
print('X_test.shape:', X_test.shape, sep='\t')

### Load Data
---

X.shape:	(29, 6)
X_train.shape:	(17, 6)
X_test.shape:	(12, 6)


In [3]:
display(Markdown('### Fit Data\n---'))
pipe.fit(X_train, y_train)
y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)
print('RMSE train: %.3f, test: %.3f' % (np.sqrt(mean_squared_error(y_train, y_train_pred)), np.sqrt(mean_squared_error(y_test, y_test_pred))))
print('R^2 train: %.3f, test: %.3f' % (r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)))
scores = cross_val_score(pipe, X, y, scoring=make_scorer(mean_absolute_error), cv=5)
print('Mean and std of CV MAE:', np.mean(scores), np.std(scores))

### Fit Data
---

RMSE train: 0.572, test: 0.693
R^2 train: 0.996, test: 0.995
Mean and std of CV MAE: 3.978708898909887 2.1380060781855637


In [4]:
display(Markdown('### Export Estimator\n---'))
sqf_code = export_estimator(pipe, "_test_estimator")
Markdown(f'```c++\n{sqf_code}\n```')

### Export Estimator
---

```c++
_test_estimator_polyfeat = {[
    1, (_this#0), (_this#1), (_this#2), (_this#3), (_this#4), (_this#5),
    (_this#0)^2, (_this#0)*(_this#1), (_this#0)*(_this#2), (_this#0)*(_this#3),
    (_this#0)*(_this#4), (_this#0)*(_this#5), (_this#1)^2, (_this#1)*(_this#2),
    (_this#1)*(_this#3), (_this#1)*(_this#4), (_this#1)*(_this#5), (_this#2)^2,
    (_this#2)*(_this#3), (_this#2)*(_this#4), (_this#2)*(_this#5), (_this#3)^2,
    (_this#3)*(_this#4), (_this#3)*(_this#5), (_this#4)^2, (_this#4)*(_this#5),
    (_this#5)^2]
};

_test_estimator_linreg = {
    4.480465e-17*(_this#0) - 1.512462e-17*(_this#1) - 1.192622e-18*(_this#2) +
    4.770490e-18*(_this#3) + 4.218835e-07*(_this#4) + 2.880030e-07*(_this#5) +
    6.561057e-08*(_this#6) - 8.131516e-20*(_this#7) - 1.084202e-19*(_this#8) +
    3.081488e-33*(_this#9) + 7.551714e-04*(_this#10) + 5.155254e-04*(_this#11) +
    1.174429e-04*(_this#12) + 2.311116e-33*(_this#13) + 4.237046e-33*(_this#14)
    + 5.526674e-04*(_this#15) + 3.772839e-04*(_this#16) +
    8.594984e-05*(_this#17) + 8.015786e-06*(_this#19) + 5.472057e-06*(_this#20)
    + 1.246601e-06*(_this#21) - 6.540086e-04*(_this#22) +
    2.028522e-04*(_this#23) - 1.568895e-05*(_this#24) - 6.650353e-04*(_this#25)
    - 2.561449e-04*(_this#26) + 3.550490e-04*(_this#27) - 2.726249e+03
};

_test_estimator = {
    _this = _this call _test_estimator_polyfeat;
    _this = _this call _test_estimator_linreg;
    _this
};

```

In [5]:
help(export_estimator)

Help on function export_estimator in module rega3.exporter:

export_estimator(estimator, function_name, decimal_places=6, indent='    ', line_length=80)
    Export trained regression model from Scikit-learn to SQF code.
    
    :param estimator: Fitted estimator to export_estimator.
    :type estimator: sklearn.base.BaseEstimator
    :param function_name: Name for the SQF function.
    :type function_name: str
    :param decimal_places: Number of decimal places for coefficients.
        Defaults to 6.
    :type decimal_places: int, optional
    :param indent: The string used for indentations. Defaults to four spaces.
    :type indent: str, optional
    :param line_length: Maximum line length for the code. Defaults to 80.
    :type line_length: int, optional

